In [ ]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel, CLIPFeatureExtractor,CLIPVisionModel,CLIPVisionConfig

from torchvision.datasets import CIFAR100, ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as T
import os

In [ ]:

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
#cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [ ]:
transformes = T.Compose([T.Resize(255),
                                T.CenterCrop(224),
                                T.ToTensor()])

ims = ImageFolder("//home/ben/Downloads/factory_doran/",transform=transformes)
dataloader = torch.utils.data.DataLoader(ims, batch_size=1, shuffle=True)

In [ ]:
image_cart = []
image_conc = []
image_fact = []
image_mach = []
image_rand = []
image_test = []

for image, class_id in dataloader:
    if class_id==0:
        image_cart.append(image.squeeze())
    elif class_id==1:
        image_conc.append(image.squeeze())
    elif class_id==2:
        image_fact.append(image.squeeze())
    elif class_id==3:
        image_mach.append(image.squeeze())
    elif class_id==4:
        image_rand.append(image.squeeze())
    elif class_id==5:
        image_test.append(image.squeeze())


In [ ]:
import logictensornetworks as ltn
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2,stable=True),semantics="forall")

formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=2,stable=True)

In [ ]:
inputs_cart = processor(images=image_cart, return_tensors="pt", padding=True)
inputs_conc = processor(images=image_conc, return_tensors="pt", padding=True)
inputs_fact = processor(images=image_fact, return_tensors="pt", padding=True)
inputs_mach = processor(images=image_mach, return_tensors="pt", padding=True)
inputs_rand = processor(images=image_rand, return_tensors="pt", padding=True)
inputs_test = processor(images=image_test, return_tensors="pt", padding=True)

In [ ]:
for parameter in model.parameters():
    parameter.requires_grad = False

In [ ]:
feats_cart = model.get_image_features(**inputs_cart)
feats_conc = model.get_image_features(**inputs_conc)
feats_fact = model.get_image_features(**inputs_fact)
feats_mach = model.get_image_features(**inputs_mach)
feats_rand = model.get_image_features(**inputs_rand)
feats_test = model.get_image_features(**inputs_test)

In [ ]:
cart = ltn.Predicate.MLP([512],hidden_layer_sizes=[10])
conc = ltn.Predicate.MLP([512],hidden_layer_sizes=[10])
fact = ltn.Predicate.MLP([512],hidden_layer_sizes=[10])
mach = ltn.Predicate.MLP([512],hidden_layer_sizes=[10])

def axioms():
    x_A = ltn.variable("x_A",feats_cart)
    x_B = ltn.variable("x_B",feats_conc)
    x_C = ltn.variable("x_C",feats_fact)
    x_D = ltn.variable("x_D",feats_mach)
    x_R = ltn.variable("x_R",feats_rand)

    axioms = [
        Forall(x_A, cart(x_A)),
        Forall(x_R, Not(cart(x_R))),
        Forall(x_B, conc(x_B)),
        Forall(x_R, Not(conc(x_R))),
        Forall(x_C, fact(x_C)),
        Forall(x_R, Not(fact(x_R))),
        Forall(x_D, mach(x_D)),
        Forall(x_R, Not(mach(x_R))),
    ]
    axioms = torch.stack(axioms)
    sat_level = formula_aggregator(axioms, axis=0)
    return sat_level, axioms

In [ ]:
axioms()

(tensor(0.4936, grad_fn=<RsubBackward1>),
 tensor([0.4413, 0.5573, 0.4712, 0.5292, 0.4962, 0.4951, 0.4130, 0.5655],
        grad_fn=<StackBackward>))

In [ ]:
from torch.optim import Adam

trainable_variables = list(cart.parameters())\
                      +list(conc.parameters())\
                      +list(fact.parameters())\
                      +list(mach.parameters())
optimizer = Adam(trainable_variables, lr=0.001)

In [ ]:
for epoch in range(50):
    optimizer.zero_grad()
    loss_value = 1. - axioms()[0]
    loss_value.backward()
    optimizer.step()
    if epoch%10 == 0:
        print("Epoch %d: Sat Level %.3f"%(epoch, axioms()[0]))

Epoch 0: Sat Level 0.998
Epoch 10: Sat Level 0.999
Epoch 20: Sat Level 0.999
Epoch 30: Sat Level 0.999
Epoch 40: Sat Level 0.999


In [ ]:
x_A = ltn.variable("x_A",feats_cart)
x_B = ltn.variable("x_A",feats_conc)
x_test = ltn.variable("x_test", feats_test)

Forall(x_A, cart(x_A))

tensor(0.9248, grad_fn=<RsubBackward1>)

In [ ]:
Forall(x_A, conc(x_A))

tensor(0.6873, grad_fn=<RsubBackward1>)

In [ ]:
fact(x_test)

tensor([0.9699], grad_fn=<ViewBackward>)

In [ ]:
cart(x_test)

tensor([0.5433], grad_fn=<ViewBackward>)

In [ ]:
fact(x_test)

tensor([0.9999], grad_fn=<ViewBackward>)

In [ ]:
conc(x_test)

tensor([0.9768], grad_fn=<ViewBackward>)

In [ ]:
mach(x_test)

tensor([0.9121], grad_fn=<ViewBackward>)

In [ ]:
cart(x_test)


tensor([0.7623], grad_fn=<ViewBackward>)

In [ ]:
cart(x_test)

tensor([0.7733], grad_fn=<ViewBackward>)

In [ ]:
mach(x_test)


tensor([0.9979], grad_fn=<ViewBackward>)

In [ ]:
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=-1,stable=True),semantics="forall")

x_fac = ltn.variable("x_test", feats_fact)

Forall(x_fac,conc(x_fac))

tensor(0.9838, grad_fn=<RsubBackward1>)

In [ ]:

Forall(x_fac,mach(x_fac))

tensor(0.7580, grad_fn=<RsubBackward1>)

In [ ]:
conc(x_fac)

tensor([0.9989, 0.9873, 0.3530, 0.1932, 0.1671, 0.3399, 0.8151, 0.9932, 0.4081,
        0.9863, 0.9466, 0.8921, 0.5812, 0.5889, 0.8007, 0.9762, 0.5686, 0.5620,
        0.0939, 0.0296], grad_fn=<ViewBackward>)

In [ ]:
mach(x_fac)

tensor([0.9989, 0.9736, 0.9072, 0.9952, 0.9993, 0.2331, 1.0000, 0.9994, 0.9765,
        0.9998, 0.9998, 0.4997, 0.4342, 0.9999, 0.9541, 0.9937, 0.9848, 0.9954,
        0.9884, 0.9980], grad_fn=<ViewBackward>)

In [ ]:
Forall(x_fac,mach(x_fac))

tensor(0.9992, grad_fn=<RsubBackward1>)

In [ ]:
Forall(x_fac,cart(x_fac))

tensor(0.8891, grad_fn=<RsubBackward1>)

In [ ]:
Forall(x_fac,Implies(mach(x_fac),fact(x_fac)))

tensor(0.9995, grad_fn=<RsubBackward1>)

In [ ]:
Forall(x_fac,Implies(And(conc(x_fac),mach(x_fac)),fact(x_fac)))

tensor(0.9996, grad_fn=<RsubBackward1>)

In [ ]:
Forall(x_fac,mach(x_fac))

In [ ]:
Forall(x_fac,mach(x_fac))



In [ ]:
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=1,stable=True),semantics="forall")

In [ ]:
inputs_cart_t = processor(text=["A photo of boxes", "A photo of concrete floor", "A photo of a factory","A photo of a machine"],images=image_cart, return_tensors="pt", padding=True)
inputs_conc_t = processor(text=["A photo of boxes", "A photo of concrete floor", "A photo of a factory","A photo of a machine"],images=image_conc, return_tensors="pt", padding=True)
inputs_fact_t = processor(text=["A photo of boxes", "A photo of concrete floor", "A photo of a factory","A photo of a machine"],images=image_fact, return_tensors="pt", padding=True)
inputs_mach_t = processor(text=["A photo of boxes", "A photo of concrete floor", "A photo of a factory","A photo of a machine"],images=image_mach, return_tensors="pt", padding=True)
inputs_test_t = processor(text=["A photo of boxes", "A photo of concrete floor", "A photo of a factory","A photo of a machine","A photo of a dog"],images=image_test, return_tensors="pt", padding=True)

In [ ]:
outs = model(**inputs_test_t)

In [ ]:
outs.logits_per_image.shape
soft = outs.logits_per_image.softmax(dim=1)
soft.shape

torch.Size([1, 4])

In [ ]:
outs.logits_per_image

tensor([[21.8592, 22.2974, 27.8345, 24.8549, 25.5312]])

In [ ]:
from torch import nn

class LTNextract(nn.Module):
    def __init__(self, index):
        super(LTNextract,self).__init__()
        self.index = index

    def forward(self, inputs):
        soft = inputs.softmax(dim=1)
        output = soft[:,self.index]
        return output

In [ ]:
cart_t = ltn.Predicate(LTNextract(0))
conc_t = ltn.Predicate(LTNextract(1))
fact_t = ltn.Predicate(LTNextract(2))
mach_t = ltn.Predicate(LTNextract(3))
def axioms():
    x_A = ltn.variable("x_A",outs.logits_per_image)
    # x_B = ltn.variable("x_B",outs.logits_per_image)
    axioms = [
        Forall(x_A, cart_t(x_A)),
        # Forall(x_B, Not(apps(x_B))),
        # Forall(x_A, Not(crabs(x_A))),
        Forall(x_A, conc_t(x_A)),
        Forall(x_A, fact_t(x_A)),
        Forall(x_A, mach_t(x_A))
    ]
    axioms = torch.stack(axioms)
    sat_level = formula_aggregator(axioms, axis=0)
    return sat_level, axioms

In [ ]:
# PYDEVD_USE_CYTHON = "YES"
axioms()

(tensor(0.1490), tensor([0.0024, 0.0037, 0.9457, 0.0481]))

In [ ]:
outs.logits_per_text[0].shape
LTNextract(outs.logits_per_text)

LTNextract()

In [ ]:
x_A = ltn.variable("x_A",outs.logits_per_text)
x_A.softmax(dim=1)[0]

In [ ]:
x_A[0]

In [ ]:
Forall(x_A, x_A.softmax(dim=1)[0])


In [ ]:
test = LTNextract(1)
test(outs.logits_per_text)

In [ ]:
outs.logits_per_image.softmax(dim=1)[0]

In [ ]:
# DERIVE ANTON YMS/DISTINCT FROM CONCEPTNET
inputs_test_t_cart = processor(text=["A photo of boxes", "A photo of cylinders", "A photo of a bin","A photo of papers"],images=image_test, return_tensors="pt", padding=True)
inputs_test_t_conc = processor(text=["A photo of concrete floor", "A photo of wooden floor", "A photo of tiled floor","A photo of linoleum floor"],images=image_test, return_tensors="pt", padding=True)
inputs_test_t_fact = processor(text=["A photo of a factory","A photo of an artist", "A photo of a company", "A photo of a farm"],images=image_test, return_tensors="pt", padding=True)
inputs_test_t_mach = processor(text=["A photo of a machine", "A photo of a human", "A photo of a people","A photo of organic things"],images=image_test, return_tensors="pt", padding=True)

In [ ]:
outs_cart = model(**inputs_test_t_cart)
outs_conc = model(**inputs_test_t_conc)
outs_fact = model(**inputs_test_t_fact)
outs_mach = model(**inputs_test_t_mach)


torch.Size([1, 4])

In [ ]:
outs_conc.logits_per_image.shape
soft = outs_conc.logits_per_image.softmax(dim=1)
soft

tensor([[0.3778, 0.0308, 0.0341, 0.5573]])

In [ ]:
outs_conc.logits_per_image

tensor([[22.2974, 19.7899, 19.8936, 22.6863]])

In [ ]:
from torch import nn

class LTNextract(nn.Module):
    def __init__(self, index):
        super(LTNextract,self).__init__()
        self.index = index

    def forward(self, inputs):
        soft = inputs.softmax(dim=1)
        output = soft[:,self.index]
        return output

In [ ]:
concept = ltn.Predicate(LTNextract(0))
c_adv1 = ltn.Predicate(LTNextract(1))
c_adv2 = ltn.Predicate(LTNextract(2))
c_adv3 = ltn.Predicate(LTNextract(3))

def axioms():
    x_A = ltn.variable("x_A",outs_conc.logits_per_image)
    # x_B = ltn.variable("x_B",outs.logits_per_image)
    axioms = [
        Forall(x_A, concept(x_A)),
        # Forall(x_B, Not(apps(x_B))),
        # Forall(x_A, Not(crabs(x_A))),
        Forall(x_A, Not(c_adv1(x_A))),
        Forall(x_A, Not(c_adv2(x_A))),
        Forall(x_A, Not(c_adv3(x_A)))
    ]
    axioms = torch.stack(axioms)
    sat_level = formula_aggregator(axioms, axis=0)
    return sat_level, axioms

In [ ]:
# PYDEVD_USE_CYTHON = "YES"
axioms()

(tensor(0.5816), tensor([0.3777, 0.9691, 0.9658, 0.4427]))

In [ ]:
def axioms():
    x_A = ltn.variable("x_A",outs_cart.logits_per_image)
    # x_B = ltn.variable("x_B",outs.logits_per_image)
    axioms = [
        Forall(x_A, concept(x_A)),
        # Forall(x_B, Not(apps(x_B))),
        # Forall(x_A, Not(crabs(x_A))),
        Forall(x_A, Not(c_adv1(x_A))),
        Forall(x_A, Not(c_adv2(x_A))),
        Forall(x_A, Not(c_adv3(x_A)))
    ]
    axioms = torch.stack(axioms)
    sat_level = formula_aggregator(axioms, axis=0)
    return sat_level, axioms
axioms()

(tensor(0.6336), tensor([0.4013, 0.7105, 0.9985, 0.6921]))

In [ ]:
def axioms():
    x_A = ltn.variable("x_A",outs_fact.logits_per_image)
    # x_B = ltn.variable("x_B",outs.logits_per_image)
    axioms = [
        Forall(x_A, concept(x_A)),
        # Forall(x_B, Not(apps(x_B))),
        # Forall(x_A, Not(crabs(x_A))),
        Forall(x_A, Not(c_adv1(x_A))),
        Forall(x_A, Not(c_adv2(x_A))),
        Forall(x_A, Not(c_adv3(x_A)))
    ]
    axioms = torch.stack(axioms)
    sat_level = formula_aggregator(axioms, axis=0)
    return sat_level, axioms
axioms()


(tensor(0.9694), tensor([0.9567, 0.9998, 0.9568, 0.9999]))

In [ ]:
def axioms():
    x_A = ltn.variable("x_A",outs_mach.logits_per_image)
    # x_B = ltn.variable("x_B",outs.logits_per_image)
    axioms = [
        Forall(x_A, concept(x_A)),
        # Forall(x_B, Not(apps(x_B))),
        # Forall(x_A, Not(crabs(x_A))),
        Forall(x_A, Not(c_adv1(x_A))),
        Forall(x_A, Not(c_adv2(x_A))),
        Forall(x_A, Not(c_adv3(x_A)))
    ]
    axioms = torch.stack(axioms)
    sat_level = formula_aggregator(axioms, axis=0)
    return sat_level, axioms
axioms()

(tensor(0.9938), tensor([0.9895, 0.9962, 0.9954, 0.9978]))

In [ ]:
inputs_test_t_cart_alt = processor(text=["A photo containing boxes", "A photo containing cylinders", "A photo containing bin","A photo containing cans"],images=image_test, return_tensors="pt", padding=True)
outs_cart_alt = model(**inputs_test_t_cart_alt)
def axioms():
    x_A = ltn.variable("x_A",outs_cart_alt.logits_per_image)
    # x_B = ltn.variable("x_B",outs.logits_per_image)
    axioms = [
        Forall(x_A, concept(x_A)),
        # Forall(x_B, Not(apps(x_B))),
        # Forall(x_A, Not(crabs(x_A))),
        Forall(x_A, Not(c_adv1(x_A))),
        Forall(x_A, Not(c_adv2(x_A))),
        Forall(x_A, Not(c_adv3(x_A)))
    ]
    axioms = torch.stack(axioms)
    sat_level = formula_aggregator(axioms, axis=0)
    return sat_level, axioms
axioms()

(tensor(0.8739), tensor([0.7907, 0.8832, 0.9837, 0.9237]))

In [ ]:
inputs_test_f_cart = processor(text=["A photo containing boxes", "A photo containing cylinders", "A photo containing bin","A photo containing cans"],images=image_fact, return_tensors="pt", padding=True)
inputs_test_f_conc = processor(text=["A photo of concrete floor", "A photo of wooden floor", "A photo of tiled floor","A photo of linoleum floor"],images=image_fact, return_tensors="pt", padding=True)
inputs_test_f_fact = processor(text=["A photo of a factory","A photo of an artist", "A photo of a company", "A photo of a farm"],images=image_fact, return_tensors="pt", padding=True)
inputs_test_f_mach = processor(text=["A photo of a machine", "A photo of a human", "A photo of a people","A photo of organic things"],images=image_fact, return_tensors="pt", padding=True)

outs_cart = model(**inputs_test_f_cart)
outs_conc = model(**inputs_test_f_conc)
outs_fact = model(**inputs_test_f_fact)
outs_mach = model(**inputs_test_f_mach)

x_A = ltn.variable("x_A",outs_cart.logits_per_image)
x_B = ltn.variable("x_A",outs_conc.logits_per_image)
x_C = ltn.variable("x_A",outs_fact.logits_per_image)
x_D = ltn.variable("x_A",outs_mach.logits_per_image)

Forall((x_A,x_B,x_C,x_D),Implies(And(And(concept(x_A),concept(x_B)),concept(x_D)),concept(x_C)))

tensor(0.9707)

In [ ]:
Forall((x_A,x_B,x_C,x_D),Implies(concept(x_C),And(And(concept(x_A),concept(x_B)),concept(x_D))))

tensor(0.3106)

In [ ]:
def axioms():
    x_A = ltn.variable("x_A",outs_cart_alt.logits_per_image)
    x_B = ltn.variable("x_A",outs_mach.logits_per_image)
    x_C = ltn.variable("x_A",outs_fact.logits_per_image)
    x_D = ltn.variable("x_A",outs_conc.logits_per_image)
    # x_B = ltn.variable("x_B",outs.logits_per_image)
    axioms = [
        Forall(x_A, concept(x_A)),
        # Forall(x_B, Not(apps(x_B))),
        # Forall(x_A, Not(crabs(x_A))),
        Forall(x_A, Not(c_adv1(x_A))),
        Forall(x_A, Not(c_adv2(x_A))),
        Forall(x_A, Not(c_adv3(x_A)))
    ]
    axioms = torch.stack(axioms)
    sat_level = formula_aggregator(axioms, axis=0)
    return sat_level, axioms
axioms()

In [ ]:
x_A = ltn.variable("x_A",outs_conc.logits_per_text)
x_A.softmax(dim=0)

In [ ]:
x_A[0]

In [ ]:
Forall(x_A, x_A.softmax(dim=1)[0])


In [ ]:
test = LTNextract(1)
test(outs.logits_per_text)

In [ ]:
outs.logits_per_image.softmax(dim=1)[0]